## Model R1

Model: jednovstupový

Data: Close price, 2007 - duben 2019

Dataset: aapl_data_2007_2019


###Sestavení modelu

Import potřebných knihoven

In [0]:
from numpy import concatenate
from numpy import sqrt
from matplotlib import pyplot
from pandas import read_csv
from pandas import read_excel
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM

Funkce umožňující přeměnu dat do formátu potřebného pro LSTM síť

In [0]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# Vstup
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# predikce - formát (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# spojení dat do souboru
	agg = concat(cols, axis=1)
	agg.columns = names
	# odstranění prázdných proměnných v začátku dat
	if dropnan:
		agg.dropna(inplace=True)
	return agg

Import dat do prostředí Google Colab

In [0]:
dataset = read_excel('zde odkaz na dataset', header=0, index_col=0)
dataset = dataset[['Close']]

values = dataset.values

Přeměa formátu DataFrame na hodnoty, normalizace hodnot

In [0]:
values = values.astype('float32')

# normalizace provedena pomocí MinMaxScaler funkce a přeměna dat

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 5, 2)
reframed = DataFrame(reframed)


Určení Train/validation/test split

In [0]:
#Časy: 1.1.2008 - 252, konec 2015 - 2266, konec 2017 - 2769 - při 60
# split into train and test sets
#Čas: 246: 1.1.2008, 2514 - konec 2016

train = reframed.values[248:2515, :]
validation = reframed.values[2515:2766, :]

Oddělení příkladů pro učení od výsledných hodnot - X,Y
Reshape na vhodný input

In [0]:
train_X, train_y = train[:, :-1], train[:, -1]
validation_X, validation_y = validation[:, :-1], validation[:, -1]

In [0]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
validation_X = validation_X.reshape((validation_X.shape[0], 1, validation_X.shape[1]))
print(train_X.shape, train_y.shape, validation_X.shape, validation_y.shape)

(2267, 1, 6) (2267,) (251, 1, 6) (251,)


Architektura LSTM sítě

In [0]:
model = Sequential()
model.add(LSTM(200, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(Dense(1))
model.compile(loss='mae', optimizer='SGD')
model.summary()

Spuštění trnování sítě na určených datech

In [0]:
history = model.fit(train_X, train_y, epochs=250, batch_size=110, validation_data=(validation_X, validation_y), verbose=2, shuffle=False)

Graf fáze učení se neuronové sítě

In [0]:
pyplot.figure(figsize=(10,6))
pyplot.plot(history.history['loss'], label='Train set', color = '#18572D')
pyplot.plot(history.history['val_loss'], label='Validation set', color = '#FF9000')
pyplot.legend()
pyplot.xlabel('Epochs')
pyplot.ylabel('Test/Validation Loss')
pyplot.show()

Nasazení na Validation data

In [0]:
yhat_validation = model.predict(validation_X)

Přeměna dat zpět na původní tvar

In [0]:
# Přeměna zpět
validation_X = validation_X.reshape((validation_X.shape[0], validation_X.shape[2]))

In [0]:

inv_yhat_validation = concatenate((yhat_validation, validation_X[:,1:]), axis=1)

inv_yhat_validation = scaler.inverse_transform(inv_yhat_validation)
inv_yhat_validation = inv_yhat_validation[:,0]

In [0]:
# převrácená normalizac dat
validation_y = validation_y.reshape((len(validation_y), 1))
inv_y_validation = concatenate((validation_y, validation_X[:, 1:]), axis=1)
inv_y_validation = scaler.inverse_transform(inv_y_validation)
inv_y_validation = inv_y_validation[:,0]

Výpočet RMSE na validačních datech

In [0]:
# cRMSE výpočet
rmse = sqrt(mean_squared_error(inv_y_validation, inv_yhat_validation))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.377


Zobrazení na grafu

In [0]:
pyplot.figure(figsize=(15,6))
pyplot.plot(inv_y_validation, color = '#18572D')
pyplot.plot(inv_yhat_validation, color = '#FF9000')
pyplot.legend(loc="upper left")
pyplot.xlabel('Time')
pyplot.ylabel('DIS Price/Predicted price')
pyplot.title('DIS Predicted Price')
pyplot.show()

### Predikce na nových datech

In [0]:
#Určení nových dat
test = reframed.values[2763:, :]

In [0]:
#Příprava, jen hodnoty X, y neznám
test_X = test[:, :-1]

In [0]:
#reshaping
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(test_X.shape)

(312, 1, 6)


Predikování na natrénovaném modelu

In [0]:
#Predikce zde
yhat = model.predict(test_X)

Přeměna tvaru predikovaných hodnot na původní škálu a zobrazení na grafu

In [0]:
# Tohle háže test zpět do původního shape
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
inv_yhat = concatenate((yhat, test_X[:,1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

pyplot.plot(inv_yhat)

In [0]:
real_data = dataset[['Close']].iloc[2769:].values

In [0]:
#Tvar dat
real_data.shape

Graf zobrazující reálné hodnoty a hodnoty predikované na nových datech

In [0]:
pyplot.figure(figsize=(15,6))
pyplot.plot(inv_yhat, color = '#FF9000')
pyplot.plot(real_data, color = '#18572D' )
pyplot.xlabel('Time')
pyplot.ylabel('KO Price/Predicted price')
pyplot.title('KO Predicted Price')
pyplot.show()

Výpočet RMSE - nová data

In [0]:
# RMSE
rmse = sqrt(mean_squared_error(real_data, inv_yhat))
print('Test RMSE: %.3f' % rmse)

###Export výsledku do Excelu

In [0]:
Vysledek = DataFrame(inv_yhat)
Vysledek.to_excel('/content/KO_R1_Vysledek.xls')